In [1]:
from pyalgotrade import eventprofiler
from pyalgotrade.technical import stats
from pyalgotrade.technical import roc
from pyalgotrade.technical import ma
from pyalgotrade.cn import csvfeed


In [3]:
class BuyOnGap(eventprofiler.Predicate):
    def __init__(self, feed):
        stdDevPeriod = 90
        smaPeriod = 20
        self.returns = {}
        self.stdDev = {}
        self.ma = {}
        for instrument in feed.getRegisteredInstruments():
            priceDS = feed[instrument].getAdjCloseDataSeries()
            # Returns over the adjusted close values.
            self.returns[instrument] = roc.RateOfChange(priceDS, 1)
            # StdDev over those returns.
            self.stdDev[instrument] = stats.StdDev(self.returns[instrument], stdDevPeriod)
            # MA over the adjusted close values.
            self.ma[instrument] = ma.SMA(priceDS, smaPeriod)

    def gappedDown(self, instrument, bards):
        ret = False
        if self.stdDev[instrument][-1] is not None:
            prevBar = bards[-2]
            currBar = bards[-1]
            low2OpenRet = (currBar.getOpen(True) - prevBar.getLow(True)) / float(prevBar.getLow(True))
            if low2OpenRet < (self.returns[instrument][-1] - self.stdDev[instrument][-1]):
                ret = True
        return ret

    def aboveSMA(self, instrument, bards):
        ret = False
        if self.ma[instrument][-1] is not None and bards[-1].getOpen(True) > self.ma[instrument][-1]:
            ret = True
        return ret

    def eventOccurred(self, instrument, bards):
        ret = False
        if self.gappedDown(instrument, bards) and self.aboveSMA(instrument, bards):
            ret = True
        return ret

In [6]:
roc.RateOfChange?

2017-08-17 16:12:22,105 yahoofinance [INFO] Downloading AA 2008 to .\AA-2008-yahoofinance.csv


ConnectionError: HTTPConnectionPool(host='ichart.finance.yahoo.com', port=80): Max retries exceeded with url: /table.csv?s=AA&a=0&b=1&c=2008&d=11&e=31&f=2008&g=d&ignore=.csv (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x0000000007B77D30>: Failed to establish a new connection: [Errno 11004] getaddrinfo failed',))

In [3]:
import os
os.chdir(r'C:\Users\linner\Desktop\learn\pyalgo\example\data')
feed=csvfeed.Feed(60*60*24)
feed.loadBars('aig','sh','20080201','20080706','AIG-2008-yahoofinance.csv')
feed.loadBars('aapl','sh','200110201','200110701','aapl-2011-yahoofinance.csv')

KeyError: 'datetime'

In [18]:
dir(feed)

['_BaseFeed__ds',
 '_BaseFeed__event',
 '_BaseFeed__maxLen',
 '_BaseFeed__rowFilter',
 '_BaseFeed__rowParser',
 '_Feed__rowParser',
 '_MemFeed__nextIdx',
 '_MemFeed__values',
 '_Subject__dispatchPrio',
 '__abstractmethods__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__getitem__',
 '__hash__',
 '__init__',
 '__iter__',
 '__metaclass__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 'addValues',
 'addValuesFromCSV',
 'createDataSeries',
 'dispatch',
 'eof',
 'getDispatchPriority',
 'getKeys',
 'getNewValuesEvent',
 'getNextValues',
 'getNextValuesAndUpdateDS',
 'join',
 'onDispatcherRegistered',
 'peekDateTime',
 'registerDataSeries',
 'reset',
 'setDateRange',
 'setDispatchPriority',
 'setRowFilter',
 'setTimeDelta',
 'start'

In [17]:
predicate = BuyOnGap(feed)
eventProfiler = eventprofiler.Profiler(predicate, 5, 5)
eventProfiler.run(feed, True)

results = eventProfiler.getResults()
print "%d events found" % (results.getEventCount())

AttributeError: 'Feed' object has no attribute 'getRegisteredInstruments'